# Step by Step Config

## LIBRARIES

In [1]:
from argparse import ArgumentParser
import json
import time
import pandas as pd
import tensorflow as tf
import numpy as np
import math
from decimal import Decimal
import matplotlib.pyplot as plt
from agents.ornstein_uhlenbeck import OrnsteinUhlenbeckActionNoise

## Variables

In [2]:
eps=10e-8 # Smoothing term to avoid division by zero
epochs=0 # Number of epochs
M=0 # Number of assets

## MAIN

In [3]:
# # def build_parser():
# parser = ArgumentParser(description='Provide arguments for training different DDPG or PPO models in Portfolio Management')
# parser.add_argument("--mode",dest="mode",help="download(China), train, test",metavar="MODE", default="train",required=True)
# parser.add_argument("--model",dest="model",help="DDPG,PPO",metavar="MODEL", default="DDPG",required=False)

# args=vars(parser.parse_args())
# print(args)

In [4]:
args={'mode': 'test', 'model': 'PPO'}

In [5]:
with open('config.json') as f:
        config=json.load(f)

In [6]:
mode=args['mode']

def parse_config(config,mode):
    codes = config["session"]["codes"]
    start_date = config["session"]["start_date"]
    end_date = config["session"]["end_date"]
    features = config["session"]["features"]
    agent_config = config["session"]["agents"]
    market = config["session"]["market_types"]
    noise_flag, record_flag, plot_flag=config["session"]["noise_flag"],config["session"]["record_flag"],config["session"]["plot_flag"]
    predictor, framework, window_length = agent_config
    reload_flag, trainable=config["session"]['reload_flag'],config["session"]['trainable']
    method=config["session"]['method']

    global epochs
    epochs = int(config["session"]["epochs"])

    if mode=='test':
        record_flag='True'
        noise_flag='False'
        plot_flag='True'
        reload_flag='True'
        trainable='False'
        method='model_free'

    print("*--------------------Training Status-------------------*")
    print('Codes:',codes)
    print("Date from",start_date,' to ',end_date)
    print('Features:',features)
    print("Agent:Noise(",noise_flag,')---Recoed(',noise_flag,')---Plot(',plot_flag,')')
    print("Market Type:",market)
    print("Predictor:",predictor,"  Framework:", framework,"  Window_length:",window_length)
    print("Epochs:",epochs)
    print("Trainable:",trainable)
    print("Reloaded Model:",reload_flag)
    print("Method",method)
    print("Noise_flag",noise_flag)
    print("Record_flag",record_flag)
    print("Plot_flag",plot_flag)


    return codes,start_date,end_date,features,agent_config,market,predictor, framework, window_length,noise_flag, record_flag, plot_flag,reload_flag,trainable,method


codes, start_date, end_date, features, agent_config, market,predictor, framework, window_length,noise_flag, record_flag, plot_flag,reload_flag,trainable,method=parse_config(config,mode)


*--------------------Training Status-------------------*
Codes: ['AAPL', 'MSFT', 'BABA', 'SNE', 'V']
Date from 2015-01-05  to  2017-01-01
Features: ['close']
Agent:Noise( False )---Recoed( False )---Plot( True )
Market Type: America
Predictor: CNN   Framework: PPO   Window_length: 3
Epochs: 10000
Trainable: False
Reloaded Model: True
Method model_free
Noise_flag False
Record_flag True
Plot_flag True


In [7]:
from data.environment import Environment

# Create the Environment object
env = Environment(start_date, end_date, codes, features, int(window_length), market)


c:\Users\Ale\OneDrive\Escritorio\ICAI\TFG BA\XDRL for finance\XDRL-for-finance\data\environment.py:51: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  asset_data['close']=asset_data['close'].fillna(method='pad')
c:\Users\Ale\OneDrive\Escritorio\ICAI\TFG BA\XDRL for finance\XDRL-for-finance\data\environment.py:91: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  asset_data=asset_data.fillna(method='bfill',axis=1)
c:\Users\Ale\OneDrive\Escritorio\ICAI\TFG BA\XDRL for finance\XDRL-for-finance\data\environment.py:92: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  asset_data=asset_data.fillna(method='ffill',axis=1)#根据收盘价填充其他值
c:\Users\Ale\OneDrive\Escritorio\ICAI\TFG BA\XDRL for finance\XDRL-for-finance\data\environment.py:92: F

*-------------Now Begin To Generate Tensor---------------*
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', 'open', 'high', 'low', 'volume', 'percent'], dtype='object')
Index(['close', '

In [23]:
def fill_zeros(x):
    return '0'*(6-len(x))+x

#preprocess parameters
cost=0.0025

#read all data
data=pd.read_csv(r'./data/'+market+'.csv',index_col=0,parse_dates=True,dtype=object)
data["code"]=data["code"].astype(str)
if market=='China':
    data["code"]=data["code"].apply(fill_zeros)

data=data.loc[data["code"].isin(codes)]
data[features]=data[features].astype(float)

# 生成有效时间
start_date = [date for date in data.index if date > pd.to_datetime(start_date)][0]
end_date = [date for date in data.index if date < pd.to_datetime(end_date)][-1]
print(start_date.strftime("%Y-%m-%d"),end_date.strftime("%Y-%m-%d"))
print(data.index)
data.sort_index(inplace=True)
data=data[start_date.strftime("%Y-%m-%d"):end_date.strftime("%Y-%m-%d")]

2015-01-06 2016-12-30
DatetimeIndex(['2015-01-05', '2015-01-06', '2015-01-07', '2015-01-08',
               '2015-01-09', '2015-01-12', '2015-01-13', '2015-01-14',
               '2015-01-15', '2015-01-16',
               ...
               '2017-12-15', '2017-12-18', '2017-12-19', '2017-12-20',
               '2017-12-21', '2017-12-22', '2017-12-26', '2017-12-27',
               '2017-12-28', '2017-12-29'],
              dtype='datetime64[ns]', name='time', length=3017, freq=None)


In [ ]:
global M
M=len(codes)+1

if framework == 'DDPG':
    print("*-----------------Loading DDPG Agent---------------------*")
    from agents.ddpg import DDPG
    agent = DDPG(predictor, len(codes) + 1, int(window_length), len(features), '-'.join(agent_config), reload_flag,trainable)

elif framework == 'PPO':
    print("*-----------------Loading PPO Agent---------------------*")
    from agents.ppo import PPO
    agent = PPO(predictor, len(codes) + 1, int(window_length), len(features), '-'.join(agent_config), reload_flag,trainable)

stocktrader=StockTrader()

if mode=='train':

    print("Training with {:d}".format(epochs))
    for epoch in range(epochs):
        print("Now we are at epoch", epoch)
        traversal(stocktrader,agent,env,epoch,noise_flag,framework,method,trainable)

        if record_flag=='True':
            stocktrader.write(epoch)

        if plot_flag=='True':
            stocktrader.plot_result()

        stocktrader.print_result(epoch,agent)
        stocktrader.reset()

elif mode=='test':
    traversal(stocktrader, agent, env, 1, noise_flag,framework,method,trainable)
    stocktrader.write(1)
    stocktrader.plot_result()
    stocktrader.print_result(1, agent)